<a href="https://colab.research.google.com/github/Param-Bhatt/NNFL-Assignment/blob/master/2/Q6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h3>Navigating to respective directory

In [ ]:
%cd "/content/drive/My Drive/NNFL-Assignments/2"
!ls

/content/drive/My Drive/NNFL-Assignments/2
assignment2.gdoc  class2_images    class_label.mat  data5.mat	  input.mat
assignment2.pdf   class3_images    data55.xlsx	    input_a2.mat  label.mat
class1_images	  class_label.csv  data5.csv	    input.gsheet  Q8.ipynb


<h3>Importing all libraries required

In [ ]:
import numpy as np 
from scipy.io import loadmat
from sklearn.preprocessing import normalize
import random
from sklearn.metrics import confusion_matrix as cm

<h3>Activation functions

In [ ]:
#Gaussian activation function
def gaussian(X, a, b):   
    K = np.zeros((X.shape[0], hidden_neurons))
    for i in range(K.shape[0]):
        for j in range(K.shape[1]):
            K[i,j] = np.exp(-b[j] * np.linalg.norm(a[:,j] - X[i,:]))  
    return K

#Tanh activation function
def tanh(X):
    return np.tanh(X)



<h3>Initialise data

In [ ]:
def init_data(data):
    X = np.array(data[ : , :-1], dtype = float)
    y = np.array(data[ : , -1], dtype = int)
    X = normalize(X, axis = 0)
    return X, y

f = loadmat('data5.mat')
D = f['x']
np.random.shuffle(D)

X_tot, y_tot = init_data(D)
X_tot = np.insert(X_tot, 0, 1, axis=1)


labels = D[:, -1]
y = np.zeros([len(X_tot), 2])

for i in range(len(labels)):
    if labels[i] == 1:
        y[i,1] = 1.0
    elif labels[i] == 0:
        y[i,0] = 1.0

hidden_neurons = 300
output_neurons = 2

<h3>K fold validation for gaussian activation

In [ ]:
%%time

print("Gaussian accuracies: ")
acc = 0
for k in range(5):

    X_train = X_tot[0 : 1718]
    y_train = y[0 : 1718]
    X_val = X_tot[1718 :]
    y_val = y[1718 :]
    
    a = np.random.rand(X_train.shape[1], hidden_neurons) 
    b = np.random.rand(hidden_neurons)
    
    # Training 
    H = gaussian(X_train, a, b)
    H_inv = np.linalg.pinv(H)
    W2 = np.matmul(H_inv, y_train)
    
    # Testing
    H_T = gaussian(X_val, a, b)
    y_pred = np.matmul(H_T, W2)
    
    print(f'Fold {k+1}:\n')
    a = [np.argmax(y_pred[i]) for i in range(len(y_pred))]
    b= [np.argmax(y_val[i]) for i in range(len(y_val))]
    confmat = cm(b,a)
    print("Confusion Matrix:\n")
    print(confmat)
    Accuracy = (confmat[0][0]+confmat[1][1])/len(y_val)
    acc += Accuracy
    Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
    Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
    print(f"Accuracy: {Accuracy}\nSensitivity: {Sensitivity}\nSpecificity: {Specificity}")
    print("=============================================")

    X_tot[0 : 430] = X_val
    X_tot[430 : ] = X_train
    y[0 : 430] = y_val
    y[430 : ] = y_train

print(f"Overall Accuracy: {acc*100/5}%")

Gaussian accuracies: 
Fold 1:

Confusion Matrix:

[[189  22]
 [ 23 196]]
Accuracy: 0.8953488372093024
Sensitivity: 0.8949771689497716
Specificity: 0.8957345971563981
Fold 2:

Confusion Matrix:

[[188  30]
 [ 21 191]]
Accuracy: 0.8813953488372093
Sensitivity: 0.9009433962264151
Specificity: 0.8623853211009175
Fold 3:

Confusion Matrix:

[[202  23]
 [ 17 188]]
Accuracy: 0.9069767441860465
Sensitivity: 0.9170731707317074
Specificity: 0.8977777777777778
Fold 4:

Confusion Matrix:

[[192  21]
 [ 16 201]]
Accuracy: 0.913953488372093
Sensitivity: 0.9262672811059908
Specificity: 0.9014084507042254
Fold 5:

Confusion Matrix:

[[179  32]
 [ 22 197]]
Accuracy: 0.8744186046511628
Sensitivity: 0.8995433789954338
Specificity: 0.8483412322274881
Overall Accuracy: 89.44186046511628%
CPU times: user 28.2 s, sys: 888 ms, total: 29.1 s
Wall time: 27.8 s


<h3>K fold validation for tanh activation

In [ ]:
%%time

y_pred_full = []
print("Tanh accuracies:\n")
acc = 0
for k in range(5):
    
    X_train = X_tot[0 : 1718]
    y_train = y[0 : 1718]
    X_val = X_tot[1718 :]
    y_val = y[1718 :]
    
    a = np.random.rand(X_train.shape[1],hidden_neurons) 
    b = np.random.rand(hidden_neurons)
    
    # Training 
    H = tanh(X_train)
    H_inv = np.linalg.pinv(H)
    W2 = np.matmul(H_inv, y_train)
    
    # Testing
    H_T = tanh(X_val)
    y_pred = np.matmul(H_T, W2)
    
    print(f'Fold {k+1}:\n')
    a = [np.argmax(y_pred[i]) for i in range(len(y_pred))]
    b= [np.argmax(y_val[i]) for i in range(len(y_val))]
    confmat = cm(b,a)
    print("Confusion Matrix:\n")
    print(confmat)
    Accuracy = (confmat[0][0]+confmat[1][1])/len(y_val)
    acc += Accuracy
    Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
    Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
    print(f"Accuracy: {Accuracy}\nSensitivity: {Sensitivity}\nSpecificity: {Specificity}")
    print("=============================================")
    

    X_tot[0 : 430] = X_val
    X_tot[430 : ] = X_train
    y[0 : 430] = y_val
    y[430 : ] = y_train
print(f"Overall Accuracy: {acc*100/5}%")

Tanh accuracies:

Fold 1:

Confusion Matrix:

[[185  33]
 [ 39 173]]
Accuracy: 0.9325581395348836
Sensitivity: 0.8160377358490566
Specificity: 0.8486238532110092
Fold 2:

Confusion Matrix:

[[186  38]
 [ 32 174]]
Accuracy: 0.9372093023255814
Sensitivity: 0.8446601941747572
Specificity: 0.8303571428571429
Fold 3:

Confusion Matrix:

[[182  31]
 [ 20 197]]
Accuracy: 0.9813953488372092
Sensitivity: 0.9078341013824884
Specificity: 0.8544600938967136
Fold 4:

Confusion Matrix:

[[168  44]
 [ 38 180]]
Accuracy: 0.9093023255813953
Sensitivity: 0.8256880733944955
Specificity: 0.7924528301886793
Fold 5:

Confusion Matrix:

[[185  33]
 [ 39 173]]
Accuracy: 0.9325581395348836
Sensitivity: 0.8160377358490566
Specificity: 0.8486238532110092
Overall Accuracy: 93.86046511627907%
CPU times: user 134 ms, sys: 78.5 ms, total: 213 ms
Wall time: 111 ms
